<a href="https://colab.research.google.com/github/kevinvbc/gorilla/blob/main/Gorilla_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gorilla Data Engineer Assignment

In [3]:
import pandas as pd
import numpy as np
import random
from itertools import product
import timeit


In [4]:
# get data from repo
!wget "https://raw.githubusercontent.com/kevinvbc/gorilla/master/gorilla_test_data.xlsx"
xls = pd.ExcelFile('gorilla_test_data.xlsx')
meter_list = pd.read_excel(xls, 'meter_list')
forecast_table = pd.read_excel(xls, 'forecast_table')
rate_table = pd.read_excel(xls, 'rate_table')

--2023-03-16 20:51:51--  https://raw.githubusercontent.com/kevinvbc/gorilla/master/gorilla_test_data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121386 (119K) [application/octet-stream]
Saving to: ‘gorilla_test_data.xlsx’

gorilla_test_data.x 100%[===================>] 118.54K  --.-KB/s    in 0.02s   

2023-03-16 20:51:51 (5.53 MB/s) - ‘gorilla_test_data.xlsx’ saved [121386/121386]



##Exploratory Data Analysis + Preprocessing


In [3]:
# EDA
forecast_table

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [6]:
# aq_max_kwh column seems to have NaNs
rate_table['aq_max_kwh'].unique()
rate_table[(rate_table['aq_max_kwh'].isna())]

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
2,2020-04-01,EA1,732000,NaN,0.2875
5,2020-04-01,EA2,732000,NaN,0.1849
8,2020-04-01,EA3,732000,NaN,0.2492
11,2020-04-01,EA4,732000,NaN,0.2889
14,2020-04-01,EM1,732000,NaN,0.2734
...,...,...,...,...,...
1127,2024-10-01,WA1,732000,NaN,0.8735
1130,2024-10-01,WA2,732000,NaN,0.5638
1133,2024-10-01,WM1,732000,NaN,0.6461
1136,2024-10-01,WM2,732000,NaN,0.7534


In [38]:
# fill NaNs with very large value for aq_max_kwh 
large_value = 5000000
rate_table['aq_max_kwh'] = rate_table['aq_max_kwh'].fillna(large_value)

# aq_max_kwh can be an integer, just like aq_min_kwh
rate_table['aq_max_kwh'] = rate_table['aq_max_kwh'].astype('int64')
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200,0.2652
1,2020-04-01,EA1,73200,732000,0.1980
2,2020-04-01,EA1,732000,5000000,0.2875
3,2020-04-01,EA2,0,73200,0.2970
4,2020-04-01,EA2,73200,732000,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000,0.4537
1136,2024-10-01,WM2,732000,5000000,0.7534
1137,2024-10-01,WM3,0,73200,0.7263
1138,2024-10-01,WM3,73200,732000,0.6109


In [31]:
# add date on which rate finishes to rate_table
rate_table['finish_date'] = rate_table.groupby(['exit_zone','aq_min_kwh'])['date'].shift(-1)
rate_table

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh,finish_date
0,2020-04-01,EA1,0,73200.0,0.2652,2020-10-01
1,2020-04-01,EA1,73200,732000.0,0.1980,2020-10-01
2,2020-04-01,EA1,732000,NaN,0.2875,2020-10-01
3,2020-04-01,EA2,0,73200.0,0.2970,2020-10-01
4,2020-04-01,EA2,73200,732000.0,0.1524,2020-10-01
...,...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537,NaT
1136,2024-10-01,WM2,732000,NaN,0.7534,NaT
1137,2024-10-01,WM3,0,73200.0,0.7263,NaT
1138,2024-10-01,WM3,73200,732000.0,0.6109,NaT


In [40]:
# join forecast and meter table to access exit_zone information
# set index and using .join() is optimal way of joining pandas dataframes
forecast_per_zone = forecast_table.set_index('meter_id').join(meter_list.set_index('meter_id'), on = "meter_id", how = "left")
forecast_per_zone = forecast_per_zone.sort_values(['date', 'exit_zone'])
forecast_per_zone



,date,kwh,aq_kwh,exit_zone
meter_id,,,,
14676236,2020-06-01,22.070768,28978,EA1
50264822,2020-06-01,278.759084,265667,NT1
88357331,2020-06-01,343.883152,484399,SE2
34509937,2020-06-01,78.611504,78324,SO1
14676236,2020-06-02,19.170720,28978,EA1
...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1
14676236,2022-10-01,37.122479,28978,EA1
50264822,2022-10-01,328.192979,265667,NT1


In [41]:
def compare_annual_quantity(row):

    ''' This function retrieves the applicable rate from the rate_table and returns it '''

    # Make a Boolean mask that adheres to the conditions of the correct rate
    mask =   (rate_table['date'] <= row['date']) & (rate_table['finish_date'] > row['date'])  & (rate_table['exit_zone'] == row['exit_zone']) & (rate_table['aq_min_kwh'] <= row['aq_kwh']) & (rate_table['aq_max_kwh'] > row['aq_kwh'])
    relevant_rows = rate_table.loc[mask]

    # Return the relevant rates
    if len(relevant_rows) == 0:
        return None
    else:
        
        return relevant_rows['rate_p_per_kwh'].squeeze()




In [42]:
# Add the relevant rates to the forecast table for each day and for each meter
forecast_per_zone['relevant_rate_p_per_kwh'] = forecast_per_zone.apply(compare_annual_quantity, axis=1).astype('float')
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate_p_per_kwh
meter_id,,,,,
14676236,2020-06-01,22.070768,28978,EA1,0.2652
50264822,2020-06-01,278.759084,265667,NT1,0.2181
88357331,2020-06-01,343.883152,484399,SE2,0.2318
34509937,2020-06-01,78.611504,78324,SO1,0.2743
14676236,2020-06-02,19.170720,28978,EA1,0.2652
...,...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1,0.4316
14676236,2022-10-01,37.122479,28978,EA1,0.4674
50264822,2022-10-01,328.192979,265667,NT1,0.3844


In [43]:
# Compute daily charge
forecast_per_zone['daily_charge'] = forecast_per_zone['relevant_rate_p_per_kwh']*forecast_per_zone['kwh']
forecast_per_zone

,date,kwh,aq_kwh,exit_zone,relevant_rate_p_per_kwh,daily_charge
meter_id,,,,,,
14676236,2020-06-01,22.070768,28978,EA1,0.2652,5.853168
50264822,2020-06-01,278.759084,265667,NT1,0.2181,60.797356
88357331,2020-06-01,343.883152,484399,SE2,0.2318,79.712115
34509937,2020-06-01,78.611504,78324,SO1,0.2743,21.563136
14676236,2020-06-02,19.170720,28978,EA1,0.2652,5.084075
...,...,...,...,...,...,...
34509937,2022-09-30,81.478131,78324,SO1,0.4316,35.165961
14676236,2022-10-01,37.122479,28978,EA1,0.4674,17.351047
50264822,2022-10-01,328.192979,265667,NT1,0.3844,126.157381


In [25]:
# Total cost and consumption per meter in pounds
total_cost_per_meter =  pd.DataFrame((forecast_per_zone['daily_charge'].groupby('meter_id').sum()*0.01).round(2))
total_cost_per_meter.rename(columns={"daily_charge": "Total_cost(pounds)"},inplace=True)
total_cost_per_meter['Total Estimated Consumption(kWh)'] = (forecast_per_zone['kwh'].groupby('meter_id').sum().round(2))
total_cost_per_meter

,Total_cost(pounds),Total Estimated Consumption(kWh)
meter_id,,
14676236,100.15,28978.0
34509937,275.49,78324.0
50264822,731.24,265667.0
88357331,1433.16,484399.0


##Random Meter Generator

In [28]:
def meter_generator(size):
  """This function takes in an amount of meters it needs to randomly generate and outputs a list of meters with annual quantities and exit_zones"""
  
  valid_exit_zones = rate_table['exit_zone'].unique().tolist()
  meter_ids = random.sample(range(10000000, 99999999), size)
  annual_quantities = random.sample(range(10000, 100000), size)
  exit_zones = random.choices(valid_exit_zones, k= size) #sample from list of valid exit zones with replacement

  meter_list = [list(x) for x in zip(meter_ids, annual_quantities, exit_zones)]

  return meter_list


In [12]:
meters = meter_generator(10)
meters

[[18977015, 30230, 'SC1'],
 [56215175, 69614, 'SC4'],
 [65464511, 15950, 'EA2'],
 [24831694, 92633, 'NE3'],
 [53418792, 81578, 'LC'],
 [84404309, 76096, 'EM3'],
 [78623525, 38726, 'SE2'],
 [16667130, 88978, 'WM3'],
 [16013958, 47480, 'LW'],
 [47034362, 85151, 'LS']]

## Mock Consumption Data Generator

In [22]:
# define start date for this dataset
forecast_start_date = forecast_table['date'].min()

def mock_consumption_generator(meter_list, start_date = forecast_start_date, duration = 852):

  """ This function takes in a list of meters , a start date and a duration in number of days and outputs mock consumption data"""

  #initialize dataframe as combination of meter_ids and dates
  datelist = pd.date_range(forecast_start_date, periods=duration).tolist()
  meter_id_list = list(list(zip(*meter_list))[0])
  df = pd.DataFrame(list(product(meter_id_list, datelist)), columns=['meter_id', 'date'])
  # create random floats for forecast per day
  size = len(meter_list)*duration
  integer_list = random.sample(range(100000, 10000000), size)
  df['kWh'] = [round(x/100000,5) for x in integer_list]


  return df


In [16]:
mock_data = mock_consumption_generator(meters)
mock_data

,meter_id,date,kWh
0,18977015,2020-06-01,61.12349
1,18977015,2020-06-02,46.02557
2,18977015,2020-06-03,67.86722
3,18977015,2020-06-04,9.77835
4,18977015,2020-06-05,70.26545
...,...,...,...
8515,47034362,2022-09-26,72.23823
8516,47034362,2022-09-27,77.36669
8517,47034362,2022-09-28,3.10705
8518,47034362,2022-09-29,23.25753


## Benchmarking transportation cost table function

In [40]:
def transportation_cost(meter_list: list, consumption_forecast) :
  
  """ This function takes a meter list and consumption forecast table in the form of a pandas dataframe and returns the consumption forecast table in a pandas dataframe """

  meter_list = pd.DataFrame(meter_list, columns = ['meter_id', 'aq_kwh','exit_zone'])
  forecast_per_zone_df = consumption_forecast.set_index('meter_id').join(meter_list.set_index('meter_id'), on = "meter_id", how = "left").reset_index()
  merge_df = rate_table.merge(forecast_per_zone_df, how="cross")

  # filter relevant rows for correct rate (date_x is start date for rate determination, date_y is consumption date)
  relevant_rate_df = merge_df[(merge_df.date_x <= merge_df.date_y) & (merge_df.finish_date > merge_df.date_y) & (merge_df.exit_zone_x == merge_df.exit_zone_y) & (merge_df.aq_min_kwh <= merge_df.aq_kwh) & (merge_df.aq_max_kwh > merge_df.aq_kwh)]
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
  #relevant_rate_df.loc[:, 'transportation_cost'] = relevant_rate_df.loc[:, 'rate_p_per_kwh'] * relevant_rate_df.loc[:, 'kWh']
  relevant_rate_df= relevant_rate_df.set_index('meter_id')
  # Total cost in pounds and consumption per meter 
  total_cost_and_consumption = pd.DataFrame((relevant_rate_df['transportation_cost'].groupby('meter_id').sum()*0.01).round(2))
  total_cost_and_consumption.rename(columns={"transportation_cost": "Total_cost(pounds)"},inplace=True)
  total_cost_and_consumption['Total Estimated Consumption(kWh)'] = (relevant_rate_df['kWh'].groupby('meter_id').sum().round(2))


  return total_cost_and_consumption





In [41]:
def run_benchmark(meters, mock_data):
    setup_code = """from __main__ import transportation_cost
    """
    statement = f"transportation_cost_test()"
    execution_time = timeit.timeit(stmt=statement,setup="from __main__ import transportation_cost_test", number=5)
    return execution_time

meter_list_sizes = [100,1000,10000,100000,1000000,10000000]
durations = [500, 1000, 2000,5000,10000,100000]
results = []



for d in durations:
  for m in meter_list_sizes:
    meters = meter_generator(10)
    
    mock_data = mock_consumption_generator(meters, d)

    def transportation_cost_test():
      transportation_cost(meters,mock_data)

    execution_time = run_benchmark(meters, mock_data)
    print("Execution time for meter size of ", m, "and duration of ", d, " was ", execution_time)
    results.append({'meter_list_size': m, 'duration': d, 'execution_time': execution_time})

execution_times_df = pd.DataFrame(results)
#df.sort_values('time', inplace=True)



<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Execution time for meter size of  100 and duration of  500  was  22.254832594000618


<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Execution time for meter size of  1000 and duration of  500  was  19.68242859699967


<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

...

<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Execution time for meter size of  1000000 and duration of  100000  was  21.701796315999673


<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']
<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Execution time for meter size of  10000000 and duration of  100000  was  21.743290974000047


<ipython-input-40-81820fcfa9a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_rate_df['transportation_cost'] = relevant_rate_df['rate_p_per_kwh'] * relevant_rate_df['kWh']


In [42]:
execution_times_df = pd.pivot(execution_times_df, index='meter_list_size', columns='duration', values='execution_time')
execution_times_df


duration,500,1000,2000,5000,10000,100000
meter_list_size,,,,,,
100,22.254833,21.014955,20.857173,21.975904,21.693637,21.661494
1000,19.682429,20.826958,20.931396,22.460273,22.877349,21.637752
10000,19.140450,21.143647,28.208594,21.838301,21.827164,22.768374
100000,20.338846,20.738964,23.185592,22.488054,21.815052,22.141337
1000000,19.590001,20.980756,22.135955,21.842979,22.257033,21.701796
10000000,19.711237,21.154672,21.764970,22.036761,22.589633,21.743291
